In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from random import shuffle

In [ ]:
df = pd.read_csv("chips.txt", header=None, names=["x", "y", "type"])
df['color'] = df['type'].map(lambda x: 'red' if x else 'blue')
df.head()

In [ ]:
plt.scatter(df['x'], df['y'], c=df['color'])
plt.show()

In [ ]:
def distance(a, b):
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

def KNN(learn, test, k):
    k = min(k, len(learn))
    res = [] 
    for p in test:
        a = [[distance(p[:2], x[:2]), x] for x in learn]
        a.sort()
        pprint(a)
        res.append([p[0], p[1], p[2], s])
    return res

In [ ]:
df.reindex(np.random.permutation(df.index))
data = list(zip(list(df['x']), list(df['y']), list(df['type'])))
l = int(len(data)*0.8)
t = len(data) - l
learn = data[:l]
test = data[t:]
KNN(learn, test, 3)